In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
from utils import *

tagger = MeCab.Tagger()

D:\anaconda3\envs\data_analysis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
input_data_path = '../data/input_data/'
output_data_path = '../data/output_data/'
outer_data_path = '../data/outer_data/'

In [3]:
with open(input_data_path+'train_aozora_data.json') as f:
    novels_json = json.load(f)

novel_id_l = []
author_l = []
for author_data in novels_json:
    for novel in author_data['novels']:
        novel_id_l.append(novel['content_id'])
        author_l.append(author_data['author_id'])

In [4]:
model_path = '../data/outer_data/chive-1.2-mc5.kv'
w2v_model = KeyedVectors.load(model_path)

In [5]:
with open(input_data_path+'aozora_contents.json', encoding='utf-8') as f:
    novel_text_dic = json.load(f)

train_l = []
i = 0
for novel_content_id, author_id in zip(novel_id_l, author_l):
    novel_content = novel_text_dic[novel_content_id].replace('\n', '').replace('\r', '').replace('\u3000', '')
    novel_word_l = doc_wakati(novel_content)
    text_vec = text_to_vec(novel_word_l, w2v_model)

    if text_vec is None:
        continue

    # print([author_id]+list(text_vec))
    train_l.append([author_id]+list(text_vec))
    print('\r'+str(i), end='')
    i+=1

train_df = pd.DataFrame(train_l, columns=['author_id']+[i for i in range(len(text_vec))])

13861

In [6]:
le = LabelEncoder()
train_df['author_id'] = le.fit_transform(train_df['author_id'])

843

In [7]:
with open(output_data_path+'train_df.pickle', 'wb') as f:
    pickle.dump(train_df, f)